In [1]:
# Macon's Bill No. 2
import numpy as np
import pandas as pd
from pathlib import Path
import stat as st
import subprocess

import matplotlib.pyplot as plt

In [ ]:
image_dir = input("Path to folder with nuclei images: ").strip("")

## Define required CellProfiler paths, then run CellProfiler
# Define the path to the CellProfiler executable (.exe)
cp_path = r"C:\Program Files (x86)\CellProfiler\CellProfiler.exe"

# Define the path to the pipeline (.cppipe)
cppipe_path = r"C:\Users\james\Documents\Yale\Bindra\Python GDA\greatest GDA.cppipe"

# Define the path to the .csv output folder
results_dir = r"C:\Users\james\Documents\Yale\Bindra\Python GDA"

# Run CellProfiler from the command line
subprocess.run([cp_path, "-c", "-r", "-p", cppipe_path, "-i", image_dir])

In [ ]:
# we ask the user for their CellProfiler output .csv that they wish to analyze, then save the path as a variable to call on later
csv_input = input("CellProfiler counts .csv file path: ")
csv_path = Path(csv_input)

# Check if the file exists to handle potential issues early
if not csv_path.is_file():
    print("Error: The specified file does not exist. Please check the path and try again.")
else:
    print("File path saved successfully.")

File path saved successfully.


In [8]:
# we convert the user's CellProfiler .csv output to a DataFrame using pandas, then we give it a haircut, shave, and shower
df_nuclei = pd.read_csv(csv_path)
df_nuclei.drop('ImageNumber', axis=1, inplace=True)
df_nuclei.columns = ['nuclei', 'well']
df_nuclei['well'] = df_nuclei['well'].str.split('_').str[0]
df_nuclei = df_nuclei[['well', 'nuclei']]
df_nuclei.to_csv(r'C:\Users\james\Documents\Yale\Bindra\Python GDA\example.csv')

# we create groupings based off of the technical triplicates for each condition; this allows us to analyze each condition separately later on
upper_vehicle = df_nuclei[df_nuclei['well'].isin(['B2', 'C2', 'D2'])]
upper_1 =       df_nuclei[df_nuclei['well'].isin(['B3', 'C3', 'D3'])]
upper_2 =       df_nuclei[df_nuclei['well'].isin(['B4', 'C4', 'D4'])]
upper_3 =       df_nuclei[df_nuclei['well'].isin(['B5', 'C5', 'D5'])]
upper_4 =       df_nuclei[df_nuclei['well'].isin(['B6', 'C6', 'D6'])]
upper_5 =       df_nuclei[df_nuclei['well'].isin(['B7', 'C7', 'D7'])]
upper_6 =       df_nuclei[df_nuclei['well'].isin(['B8', 'C8', 'D8'])]
upper_7 =       df_nuclei[df_nuclei['well'].isin(['B9', 'C9', 'D9'])]
upper_8 =       df_nuclei[df_nuclei['well'].isin(['B10', 'C10', 'D10'])]
upper_9 =       df_nuclei[df_nuclei['well'].isin(['B11', 'C11', 'D11'])]

lower_vehicle = df_nuclei[df_nuclei['well'].isin(['E2', 'F2', 'G2'])]
lower_1 =       df_nuclei[df_nuclei['well'].isin(['E3', 'F3', 'G3'])]
lower_2 =       df_nuclei[df_nuclei['well'].isin(['E4', 'F4', 'G4'])]
lower_3 =       df_nuclei[df_nuclei['well'].isin(['E5', 'F5', 'G5'])]
lower_4 =       df_nuclei[df_nuclei['well'].isin(['E6', 'F6', 'G6'])]
lower_5 =       df_nuclei[df_nuclei['well'].isin(['E7', 'F7', 'G7'])]
lower_6 =       df_nuclei[df_nuclei['well'].isin(['E8', 'F8', 'G8'])]
lower_7 =       df_nuclei[df_nuclei['well'].isin(['E9', 'F9', 'G9'])]
lower_8 =       df_nuclei[df_nuclei['well'].isin(['E10', 'F10', 'G10'])]
lower_9 =       df_nuclei[df_nuclei['well'].isin(['E11', 'F11', 'G11'])]

upper_conditions = [upper_vehicle, upper_1, upper_2, upper_3, upper_4,
                    upper_5, upper_6, upper_7, upper_8, upper_9]

lower_conditions = [lower_vehicle, lower_1, lower_2, lower_3, lower_4,
                    lower_5, lower_6, lower_7, lower_8, lower_9]

# we determine the mean for each condition and add it to the DataFrame
for condition in upper_conditions + lower_conditions:
    mean = condition['nuclei'].mean()
    df_nuclei.loc[df_nuclei['well'].isin(condition['well']), 'mean_nuclei'] = mean

# we make our means relative using corresponding vehicle means, located at [2] (B2 well) and [32] (E2 well)
for condition in upper_conditions:
    count = condition['nuclei']
    vehicle_mean = df_nuclei['mean_nuclei'].values[2]
    df_nuclei.loc[df_nuclei['well'].isin(condition['well']), 'mean_normalized'] = count / vehicle_mean

for condition in lower_conditions:
    count = condition['nuclei']
    vehicle_mean = df_nuclei['mean_nuclei'].values[32]
    df_nuclei.loc[df_nuclei['well'].isin(condition['well']), 'mean_normalized'] = count / vehicle_mean

# we determine the standard deviation for each condition's relative mean
for condition in upper_conditions + lower_conditions:
    std = df_nuclei.loc[df_nuclei['well'].isin(condition['well']), 'mean_normalized'].std()
    df_nuclei.loc[df_nuclei['well'].isin(condition['well']), 'std_relative'] = std

df_nuclei.to_csv(r'C:\Users\james\Documents\Yale\Bindra\Python GDA\example.csv')

In [ ]:
doses_input = input("Please input the ten concentrations, including 0, separated by commes: ").split(',')
doses = []
for dose in doses_input:
    doses = float(doses_input)
x = np.array(doses)
print(doses)

In [ ]:
# we ask the user for the ten concentrations of drug (including 0)
doses = input("Please input the ten concentrations, including 0, separated by commes: ").split(',')
x = np.array(doses)
print(doses)
# Assign average normalized nuclei counts to the y-axis for each condition
y1 = np.array(upper_normalized_means)
y2 = np.array(lower_normalized_means)

## Define non-linear regression for the xy-plot and estimate IC50s
# Define the 5PL function
def fivePL(x, A, B, C, D, G): # (doses, min y, Hill slope, inflection, max y, asymetry):
    return ((A-D) / (1.0 + (x / C)**B)**G) + D

# Initial guesses for parameters
params_init_5PL_y1 = [y1[np.argmin(y1)], 1, x[np.abs(y1-0.5).argmin()], y1[np.argmax(y1)], 1]  # [A, B, C, D, G]
params_init_5PL_y2 = [y2[np.argmin(y2)], 1, x[np.abs(y2-0.5).argmin()], y2[np.argmax(y2)], 1]  # [A, B, C, D, G]

# Generate x values for the fitted curves
x_plot = np.linspace(min(x), max(x), 1000)

# Use curve_fit to fit the data for y1 and y2
popt_5PL_y1, pcov_5PL_y1 = curve_fit(fivePL, x, y1, p0=params_init_5PL_y1, maxfev=10000)
popt_5PL_y2, pcov_5PL_y2 = curve_fit(fivePL, x, y2, p0=params_init_5PL_y2, maxfev=10000)

# Calculate y values for the fitted curves for y1 and y2
y_plot_5PL_y1 = fivePL(x_plot, *popt_5PL_y1)
y_plot_5PL_y2 = fivePL(x_plot, *popt_5PL_y2)

# Plot the fitted curves for y1 and y2
plt.plot(x_plot, y_plot_5PL_y1, 'b-')
plt.plot(x_plot, y_plot_5PL_y2, 'r-')

# Define the function to estimate IC50 for y1 and y2
def root_func_y1(x):
    return fivePL(x, *popt_5PL_y1) - 0.5
def root_func_y2(x):
    return fivePL(x, *popt_5PL_y2) - 0.5

# Use the dose (x) closest to y=0.5 as initials for IC50 estimates
initial_guess_y1 = x[np.abs(y1 - 0.5).argmin()]
initial_guess_y2 = x[np.abs(y2 - 0.5).argmin()]
print(initial_guess_y1)
print(initial_guess_y2)

# Estimate the IC50 for y1 and y2
IC50_y1 = scipy_root(root_func_y1, initial_guess_y1)
IC50_y2 = scipy_root(root_func_y2, initial_guess_y2)

# Calculate the ratio of IC50 estimates
IC50_value_y1 = IC50_y1.x[0]
IC50_value_y2 = IC50_y2.x[0]
IC50_ratio = IC50_value_y1 / IC50_value_y2

## Create scatter plot
# Create basic structure
plt.style.use('ggplot')
plt.xscale('log')
plt.scatter(x, y1, color = 'blue', label = str(upper_name))
plt.scatter(x, y2, color = 'red', label = str(lower_name))
plt.errorbar(x, y1, yerr = upper_sd, fmt='o', color='blue', capsize= 3)
plt.errorbar(x, y2, yerr = lower_sd, fmt='o', color='red', capsize=3)

# Annotate the plot
plt.xlabel('Concentration (M)')
plt.ylabel('Relative Cell Survival')
plt.title(str(title_name))
plt.text(0.05, 0.09, f'IC50 = {IC50_y1.x[0]:.2e}', color='blue', fontsize=10, transform=plt.gca().transAxes)
plt.text(0.05, 0.05, f'IC50 = {IC50_y2.x[0]:.2e}', color='red', fontsize=10, transform=plt.gca().transAxes)
plt.text(0.05, 0.01, f'IC50 ratio = {IC50_ratio:.1f}', color='black', fontsize=10, transform=plt.gca().transAxes)
plt.legend()
plt.show()

In [4]:
lower_means = []
for condition in lower_conditions:
    mean = condition['nuclei'].mean()
    lower_means.append(mean)